# 머신러닝 모델을 사용하여 웹 앱 만들기

NUFORC의 데이터베이스에서 얻은 데이터 셋에 대한 머신러닝 모델을 훈련하면서

- 훈련된 모델을 'Pickle'하는 법

- Flask 앰에서 모델을 사용하는 법

을 알아볼 것이다.

## 앱 만들기

앱을 만들기 전 다음과 같은 고려사항을 검토해야 한다
- 웹 앱인지 모바일 앱인지 
- 모델은 어디에 저장될 것인지
- 오프라인 동작을 지원할 것인지
- 모델을 훈련시키는데 어떤 기술이 사용될 것인지

또한 웹 브라우저에서 모델 자체를 학습 가능한 Flask웹 앱을 만들 수 있다. 이 작업은 JavaScript 컨택스트에서 TensorFlow.js를 사용하여 수행할 수 있다.

여기서는 Python 기반으로 모델을 학습할 것이기 때문에 Python에서 빌드한 웹 앱에서 읽을 수 있는 형식으로 내보내는데 필요한 단계를 살펴볼 것이다.

## 필요한 도구

작업을 위해서는 Flask와 Pickle이라는 두 가지 도구가 필요하며 둘 다 파이썬에서 실행 가능하다.

- Flask는 'micro-framework'로 정의되었다. 이것은 Python과 템플릿 엔진을 사용하여 웹 페이지를 빌드하는 웹 프레임 워크의 기본 기능을 제공한다.
- Pickle은 Python 객체 구조를 직렬화와 역직렬화하는 것이 가능한 Python 모듈이다. 모델을 'Pickle'하면 웹에서 사용할 수 있도록 구조를 직렬화하거나 병합한다.

  주의: Pickle은 안전하지 않으므로 파일을 'un-pickle' 하라는 메시지가 표시되면 주의해야 한다. pickle된 파일에는 `.pk1`이라는 접미사가 붙는다.

## 데이터 정리

[NUFORC](https://nuforc.org/)에서 수집 한 80,000 UFO 목격 데이터를 사용한다. 이 데이터에는 UFO 목격에 대한 몇 가지 흥미로운 설명이 있다.
- "한 남자가 밤에 풀밭에서 비추는 빛의 광선에서 나와 텍사스 인스트루먼트 주차장을 향해 달려갑니다"같은 긴 예제 설명이 존재한다
- "불빛이 우리를 쫓아다녔다" 같은 짧은 예제 설명이 존재한다.

[ufos.csv](https://github.com/codingalzi/ML-For-Beginners/blob/main/3-Web-App/1-Web-App/data/ufos.csv) 스프레드 시트에는 `city` `state` `country` `shape` `latitude` `longitude`같은 목격된 위치에 대한 정보가 열에 포함되어 있다.



1. 데이터를 가져온다. 

In [1]:
import pandas as pd
import numpy as np

ufos = pd.read_csv('https://raw.githubusercontent.com/codingalzi/ML-For-Beginners/main/3-Web-App/1-Web-App/data/ufos.csv')
ufos.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


2. ufos 데이터를 새로운 제목의 작은 데이터 프레임으로 변환한다. 필드에서 `Country`의 고유 값을 확인한다.

In [2]:
ufos = pd.DataFrame({'Seconds': ufos['duration (seconds)'], 'Country': ufos['country'],'Latitude': ufos['latitude'],'Longitude': ufos['longitude']})

ufos.Country.unique()

array(['us', nan, 'gb', 'ca', 'au', 'de'], dtype=object)

이제 `null`값을 삭제하고 1 ~ 60초 사이의 목격만 추출하여 처리할 데이터의 양을 줄일 수 있다.

In [3]:
ufos.dropna(inplace=True)

ufos = ufos[(ufos['Seconds'] >= 1) & (ufos['Seconds'] <= 60)]

ufos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25863 entries, 2 to 80330
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Seconds    25863 non-null  float64
 1   Country    25863 non-null  object 
 2   Latitude   25863 non-null  float64
 3   Longitude  25863 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1010.3+ KB


Scikit-learn의 `LabelEncoder` 라이브러리를 가져와 국가의 텍스트 값을 숫자로 변환한다. 해당 라이브러리는 데이터를 알파벳순으로 인코딩한다.

In [4]:
from sklearn.preprocessing import LabelEncoder

ufos['Country'] = LabelEncoder().fit_transform(ufos['Country'])

ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,3,53.200000,-2.916667
3,20.0,4,28.978333,-96.645833
14,30.0,4,35.823889,-80.253611
23,60.0,4,45.582778,-122.352222
24,3.0,3,51.783333,-0.783333


## 모델 빌드

이제 데이터를 훈련과 테스트 그룹으로 나누었기 때문에 모델을 훈련할 수 있다.

1. 학습하려는 세 가지 기능을 X 벡터로 선택하면 y벡터가

In [5]:
from sklearn.model_selection import train_test_split

Selected_features = ['Seconds','Latitude','Longitude']

X = ufos[Selected_features]
y = ufos['Country']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

2. 로지스틱 회귀를 이용하여 모델을 훈련한다.

In [6]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(classification_report(y_test, predictions))
print('Predicted labels: ', predictions)
print('Accuracy: ', accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       0.83      0.23      0.36       250
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00       131
           4       0.96      1.00      0.98      4743

    accuracy                           0.96      5173
   macro avg       0.96      0.85      0.87      5173
weighted avg       0.96      0.96      0.95      5173

Predicted labels:  [4 4 4 ... 3 4 4]
Accuracy:  0.960371157935434


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


정확도는 95%정도로 나쁘지 않다. `Country`와 `LaLatitude/Longitude`간의 상관관계가 존재한다.

제작한 모델은 그 모델에서 a를 추론할 수 있어야하므로 혁신적이진 않지만 웹 응용 프로그램에서 이 모델을 정리 후 내보낸 다음 사용하는 원시 데이터에서 학습하는 것이 좋다.

### 모델 'Pickle'

이제 모델을 'Pickle'할 차례이다. Pickle된 모델을 로드하여 시간, 위도 및 경도 값을 포함하는 샘플 데이터 배열에 대해 테스트 해본다.

In [7]:
import pickle
model_filename = 'ufo-model.pkl'
pickle.dump(model, open(model_filename,'wb'))

model = pickle.load(open('ufo-model.pkl','rb'))
print(model.predict([[50,44,-12]]))

[1]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


해당 모델은 영국의 국가 코드인 3을 반환한다.

### Flask 앱 만들기

Flask 앱을 빌드하여 모델을 호출하고 시각화 한다.
1. web-app이라는 폴더를 만든다
2. 이 폴더에서 세 개의 폴더를 만든다. 웹 앱 안 static 폴더 안 css안 templates 안에 파일이 있어야 한다.

```
web-app/
  static/
    css/
  templates/
notebook.ipynb
ufo-model.pkl
```

3. web-app 폴더에서 만드는 첫 번째 파일은 requirements.txt이다. 이 파일에는 앱에 필요한 종속성이 나열된다. requirements.txt에서 줄을 추가한다.

```
scikit-learn
pandas
numpy
flask
```

4. 이제 웹 앱으로 이동하여 이 파일을 실행한다.

```
cd web-app
```

5. 터미널 유형에서 요구 사항에 나열된 라이브러리를 설치하려면 다음을 줄이 필요하다

```
pip install -r requirements.txt
```

6. 이제 앱을 완료하기 위해 세 개의 파일을 더 만들어야 한다.
  
  - 루트에 app.py를 만든다
  - templates 디렉토리에 index.html을 만든다
  - static/css 디렉토리에 styles.css를 만든다

7. styles.css 파일을 구성한다

```css
body {
	width: 100%;
	height: 100%;
	font-family: 'Helvetica';
	background: black;
	color: #fff;
	text-align: center;
	letter-spacing: 1.4px;
	font-size: 30px;
}

input {
	min-width: 150px;
}

.grid {
	width: 300px;
	border: 1px solid #2d2d2d;
	display: grid;
	justify-content: center;
	margin: 20px auto;
}

.box {
	color: #fff;
	background: #2d2d2d;
	padding: 12px;
	display: inline-block;
}
```

8. 다음으로 index.html 파일을 구성한다.

```html
<!DOCTYPE html>
<html>
  <head>
    <meta charset="UTF-8">
    <title>🛸 UFO Appearance Prediction! 👽</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}">
  </head>

  <body>
    <div class="grid">

      <div class="box">

        <p>According to the number of seconds, latitude and longitude, which country is likely to have reported seeing a UFO?</p>

        <form action="{{ url_for('predict')}}" method="post">
          <input type="number" name="seconds" placeholder="Seconds" required="required" min="0" max="60" />
          <input type="text" name="latitude" placeholder="Latitude" required="required" />
          <input type="text" name="longitude" placeholder="Longitude" required="required" />
          <button type="submit" class="btn">Predict country where the UFO is seen</button>
        </form>

        <p>{{ prediction_text }}</p>

      </div>

    </div>

  </body>
</html>
```

이 파일에서 앱에서 제공할 변수 주변의 `{{}}` 구문을 확인한다. 경로에 대한 예측을 게시하는 `/predict`양식도 있다.

9. `app.py` 파일을 다음과 같이 추가한다.

```python
import numpy as np
from flask import Flask, request, render_template
import pickle

app = Flask(__name__)

model = pickle.load(open("./ufo-model.pkl", "rb"))


@app.route("/")
def home():
    return render_template("index.html")


@app.route("/predict", methods=["POST"])
def predict():

    int_features = [int(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    prediction = model.predict(final_features)

    output = prediction[0]

    countries = ["Australia", "Canada", "Germany", "UK", "US"]

    return render_template(
        "index.html", prediction_text="Likely country: {}".format(countries[output])
    )


if __name__ == "__main__":
    app.run(debug=True)
```

`python app.py`나 `python3 app.py`로 웹 서버를 실행하거나 로컬에서 시작하면 UFO가 발견 된 위치에 대한 질문에 답변을 얻을 수 있다.

그 전에 `app.py`의 다음 부분을 살펴본다
1. 종속성이 먼저 로드되고 앱이 시작된다.
2. 그 다음 모델을 가져온다.
3. 그 다음 index.html이 홈 경로에서 랜더링 된다.


`/predict` 경로에서 양식이 게시될 때 몇 가지 일이 발생한다.
1. 양식 변수가 수집되어 numpy 배열로 변환된다. 그 다음 모델로 전송되고 예측이 반환된다
2. 표시할 국가는 예측된 국가 코드에서 읽을 수 있는 텍스트로 다시 랜더링되고 헤당 값은 인텍스로 다시 전송되어 html 템플릿에서 랜더링된다.

Flask와 Pickle된 모델을 사용하여 이러한 방식으로 모델을 사용하는 것은 비교적 간단하다. 가장 어려운 것은 예측을 얻기 위해 모델로 보내야 하는 데이터의 모양을 이해하는 것이다. 이 모든 것은 모델이 어떻게 훈련되었는지에 달려 있다.